In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing libs
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from textwrap import wrap
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import plotly as py
import plotly.graph_objs as go
import os
py.offline.init_notebook_mode(connected = True)
#print(os.listdir("../input"))
import datetime as dt

# **COVID-19**

COVID-19 has, without exaggeration, changed the world. 

With the first reported case in late 2019, over a year later the world is still gripped by this virus.

The main symptoms of coronavirus (COVID-19) are:

* A high temperature – this means you feel hot to touch on your chest or back (you do not need to measure your temperature)

* A new, continuous cough – this means coughing a lot for more than an hour, or 3 or more coughing episodes in 24 hours (if you usually have a cough, it may be worse than usual)

* A loss or change to your sense of smell or taste – this means you've noticed you cannot smell or taste anything, or things smell or taste different to normal


Given the information available on the internet, it seems reasonable that people often head to Google to search for various symptoms if they are feeliung unwell. This is the basis for this notebook.

# **Problem statement**

# **Can we look at data from Google Trends to see when a new wave of cases or deaths is about to arise?**

I will primarily be focusing on deaths because cases are dependent upon testing levels, and this has varied wildly through the course of this pandemic globally, while deaths are recorded with much higher certainty.

An important note: **I will not be attempting to predict exact numbers** of deaths or cases. Rather, I will be investigating whether or not Google searches could act as an early warning sign for an imminent increase in cases & deaths.


 # **Why does this matter?**

This could be important for governments and local health authorities around the world, **particularly if a given country does not have adequate COVID-19 testing facilities** to be able to track the spread of the virus. 

A government or local health authority **could monitor Google searches for a variety of key words or phrases**, for instance all symptoms associated with COVID-19, and perhaps those associated with similar viruses such as Influenza, and if an increase in any of those search terms is detected, **action could be taken to mitigate further spread, and to prepare for a potential increase in hospitalisations**.



# Project Plan

The steps I plan to follow during this project are:

* Initial import of data, with brief cleaning & feature engineering

* A plot of deaths throughout the pandemic globally

* A focus on the UK's deaths

* Include Excess Deaths as a measure to provide context

* Introduce Google Search data for COVID-19 symptoms. I'll use "Loss of taste" but this could be expanded


# Inspiration

This is not an origianl peice of research, although I am not aware of it being applied specifiaclly to the United Kingom.

I found an interesting paper by Tado Jurić on 'medrxiv.org', which focused on search terms such as "PCR +Covid”, “PCR + test”, and symptoms “cough + corona”, “pneumonia + corona”; “muscle pain + corona” in Croatia. This got me thinking, and I wanted to investiagate.

https://www.medrxiv.org/content/10.1101/2021.03.12.21253452v1.full

I think this is a really interesting topic with a lot of **potential for all sorts of projects.**

# The data

I will be using various COVID-19 datasets uploaded on to Kaggle (special thanks to the contributors of those datasets), along with some data pulled from the 'Our World in Data' github (https://github.com/owid)

For Google Search terms, I will use data downloaded from Google Trends.

To understand how Google Trends works, here is a link to the FAQ page:

https://support.google.com/trends/answer/4365533?hl=en

Results from Google Trends are normalized

How is Google Trends data normalized?

Google Trends normalizes search data to make comparisons between terms easier. Search results are normalized to the time and location of a query by the following process:

* Each data point is divided by the total searches of the geography and time range it represents to compare relative popularity. Otherwise, places with the most search volume would always be ranked highest.

* The resulting numbers are then scaled on a range of 0 to 100 based on a topic’s proportion to all searches on all topics.

* Different regions that show the same search interest for a term don't always have the same total search volumes.

In [ ]:
daily = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
pop = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
latest = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')
mort = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/excess_mortality/excess_mortality.csv')
trend_june = pd.read_csv('/kaggle/input/google-trends/2020_June .csv')
trend_2020 = pd.read_csv('/kaggle/input/google-trends/2020_trend.csv')


# Correct date formats
daily["date"] = pd.to_datetime(daily["date"], format = '%Y-%m-%d')
daily['year'], daily['month'] = daily['date'].dt.year, daily['date'].dt.month

mort["date"] = pd.to_datetime(mort["date"], format = '%Y-%m-%d')
mort['year'], mort['month'] = mort['date'].dt.year, mort['date'].dt.month

trend_2020["Week"] = pd.to_datetime(trend_2020["Week"], format = '%d/%m/%Y')
trend_2020['year'], trend_2020['month'] = trend_2020['Week'].dt.year, trend_2020['Week'].dt.month

trend_june["Week"] = pd.to_datetime(trend_2020["Week"], format = '%d%m/%Y')
trend_june['year'], trend_june['month'] = trend_june['Week'].dt.year, trend_june['Week'].dt.month

mort = mort.replace([np.inf, -np.inf], np.nan)
mort = mort.fillna(0)

uk = mort[(mort['location'] == 'United Kingdom') & (mort['year'] < 2021)]

all_daily = daily.groupby(['date','date'])['new_cases_smoothed'].sum()

# latest[latest['Country/Region'].str.contains('United')]
# daily[daily['location'].str.contains('United K')]

uk_daily = daily[daily['location'] == 'United Kingdom']

# formats

background_color = 'white'
other = '#144082'
COLOR = '#323232'
plt.rcParams['text.color'] = '#323232'
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR
plt.rcParams["font.family"] = "sansserif"
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Overview

The pandemic has varied in intensity across the continents. However one can say that, as 2020 progressed, the intensity of the pandemic did too.

Most continents experienced a peak in daily deaths during late 2020 and early 2021.

I will draw attention to Europe in this plot, as this notebook will go on to focus on Google searches in the United Kingdom.

In [ ]:
temp_daily = daily.groupby(['date','continent'])['new_cases_smoothed'].sum().unstack()[:-1]

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

for i in temp_daily.columns:
    sns.lineplot(x='date',y=i,data=temp_daily,color='gray',ax=ax0)
sns.lineplot(x='date',y='Europe',data=temp_daily,color='#0f4c81',ax=ax0)


Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart,Xend-1)
ax0.set_ylabel("Daily Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=3, maxticks=12)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)

for s in ["left","bottom"]:
    ax0.spines[s].set_color(COLOR)
    

for i in temp_daily.columns:
    ax0.plot(Xend-25,temp_daily[i][-1], 'ko', markersize=5, fillstyle='full',color='gray', markeredgewidth=1.5)
    ax0.text(Xend-10,temp_daily[i][-1],i,color='gray',fontweight='light',fontsize=8, rotation=0)
    
# highlight Europe
ax0.plot(Xend-25,temp_daily['Europe'][-1], 'ko', markersize=5, fillstyle='full',color='#0f4c81', markeredgewidth=1.5)
ax0.text(Xend-10,temp_daily['Europe'][-1],'Europe',color='#0f4c81',fontweight='light',fontsize=8, rotation=0)

ax0.text(Xstart,Yend+50000,'Daily deaths by continent',fontsize=20,fontweight='bold')


plt.show()

As mentioned above, late 2020 and early 2021 have been bad across the globe.

India is experiencing a particularly sharp upsurge in both cases & deaths at the moment, contributing to the sharp increase in daily deaths observed in Asia. I continue to hope that it slows there and things improve soon.

I'll now **focus on the United Kingdom**, where currently deaths are very low, and a world leading vaccination programme seems to be making a huge difference in redcing both cases and deaths.

However, at the peak of the pandemic in the UK, in January 2021, **the UK was suffering a lot - with nearly 2,000 deaths per day.**

In [ ]:
import math
import random

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(2, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])

ax0.set_facecolor(background_color)
for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)

    
# make cloud of dots this size
num_dots = round(uk_daily['new_deaths'].max())

#x = [random.triangular() for i in range(no_of_assets)] - diff shape, more like an oval

x = [np.random.uniform() for i in range(num_dots)]
y = [random.gauss(0.5, 0.5) for i in range(num_dots)]

#random colours - alternative method
#colors = [random.randint(1, 3) for i in range(no_of_assets)]

# size of dots
areas = [math.pi * random.randint(1, 6)**1.3 for i in range(num_dots)]

def pltcolor(lst):
    cols=[]
    for l in lst:
        if l < random.triangular(0,1,0.2):
            cols.append('lightgray')
        elif l > random.triangular(0,1,0.8):
            cols.append('lightgray')
        else:
            cols.append('gray')
        
    return cols


# Create the colors list using the function above
cols=pltcolor(x)

sns.scatterplot(x, y, s=areas, c=cols, alpha=0.85,ec='black',linewidth=1.2,ax=ax0)

ax0.tick_params(axis = "both", which = "both", left=False, bottom=False,labelbottom=False, labelleft=False)


ax0.text(0.5,2.5,'Max. daily deaths in the UK',fontweight='bold',fontsize=14,ha='center')

ax0.text(0.5,-2.5,'{:,}'.format(num_dots),color='#0F4C81',fontweight='bold',fontsize=40,ha='center')

plt.show()

I'll now highlight the UK explicitly, and view how daily deaths changed over time.

You'll clearly see the '**waves**' of the pandemic experienced.

(note that these numbers have been smoothed)

In [ ]:
temp_daily = uk_daily.groupby(['date','location'])['new_deaths_smoothed'].sum().unstack()

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

sns.lineplot(x='date',y='United Kingdom',data=temp_daily,color='#0f4c81',ax=ax0)

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart,Xend-1)
ax0.set_ylabel("Daily Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=3, maxticks=12)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)

for s in ["left","bottom"]:
    ax0.spines[s].set_color(COLOR)
    
for i in temp_daily.columns:
    ax0.plot(Xend-25,temp_daily[i][-1], 'ko', markersize=5, fillstyle='full',color='#0f4c81', markeredgewidth=1.5)
    ax0.text(Xend-10,temp_daily[i][-1],i,color='#0f4c81',fontweight='light',fontsize=8, rotation=0)
    
ax0.text(Xstart,Yend+100,"Daily deaths in the UK",fontsize=20,fontweight='bold')


plt.show()

# Adding context

These patterns look frightening, but without context, it's hard to say if the pattern above is different to ordinary deaths experienced in a given year/timeframe.

A common way to deal with this problem is too introduce "excess deaths". 

Excess deaths is the amount of deaths above the average amount expected for a given time period. Below, I will look at monthly deaths in every year from 2015 to 2020, and compare that to the expected amount of the 5-year average for any given month.

In [ ]:
# data prep

uk[[ 'average_deaths_2015_2019_all_ages',
       'deaths_2015_all_ages', 'deaths_2016_all_ages', 'deaths_2017_all_ages',
       'deaths_2018_all_ages', 'deaths_2019_all_ages','deaths_2020_all_ages', 'time', 'time_unit',
       'year', 'month']]

uk['15_to_av'] = uk['deaths_2015_all_ages']/uk['average_deaths_2015_2019_all_ages']
uk['16_to_av'] = uk['deaths_2016_all_ages']/uk['average_deaths_2015_2019_all_ages']
uk['17_to_av'] = uk['deaths_2017_all_ages']/uk['average_deaths_2015_2019_all_ages']
uk['18_to_av'] = uk['deaths_2018_all_ages']/uk['average_deaths_2015_2019_all_ages']
uk['19_to_av'] = uk['deaths_2019_all_ages']/uk['average_deaths_2015_2019_all_ages']
uk['20_to_av'] = uk['deaths_2020_all_ages']/uk['average_deaths_2015_2019_all_ages']

#uk[['month','20_to_av','19_to_av','18_to_av','17_to_av','16_to_av','15_to_av']].set_index('month').T

temp = pd.pivot_table(data=uk, columns=['month','time']).T

###

fig = plt.figure(figsize=(15, 3), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])

ax0.set_facecolor(background_color)

for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)

colors = ['lightgray','gray','#0f4c81','#9b1b30']
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
    
sns.heatmap(temp[['20_to_av','19_to_av','18_to_av','17_to_av','16_to_av','15_to_av']].T,ax=ax0,linewidth=2.5,vmin=0.9,vmax=1.2, cmap=cmap,
    yticklabels=['2020','2019','2018','2017','2016','2015'],
    xticklabels=['Jan','','','','Feb','','','','Mar','','','','','Apr','','','','May','','','','','Jun','','','','Jul','','','','Aug',
                 '','','','','Sep','','','','Oct','','','','Nov','','','','','Dec','','',''],
            cbar=False,cbar_kws = dict(use_gridspec=False,location="bottom",shrink=0.2),square=True)

plt.yticks(rotation=0,fontfamily='monospace',size=12)

ax0.set_xlabel("")
ax0.set_ylabel("")
ax0.tick_params(axis=u'both', which=u'both',length=0)
ax0.xaxis.tick_top()


ax0.text(0,-3.5,'Excess deaths through the years',fontweight='bold',fontsize=25)
ax0.text(0,-2,'Shown is the deviation from the average deaths for a given month',fontsize=15)

plt.show()

# Observations

It is clear that 2020 has many months 'in the red', meaning way above the 5-year average for deaths in those particular months. Of course, other years see months above the average too, but that is to be expected.

We also see, even in 2020, months that are very pale coloured, meaning they are below the 5-year average. 

Let's now plot these explicitly, highlighting the gap between deaths suffered in 2020, and the 5-year average.

In [ ]:
fig = plt.figure(figsize=(10, 3), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)
    
temp_3 = uk.loc['2020-01-01':]

sns.lineplot(x='date',y='deaths_2020_all_ages',data=temp_3,color='#0f4c81', ax=ax0)
sns.lineplot(x='date',y='average_deaths_2015_2019_all_ages',data=temp_3,color='gray', ax=ax0)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=3, maxticks=15)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart+10,Xend-30)

ax0.set_ylabel("Daily Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)

y1 = temp_3['deaths_2020_all_ages']
y2 = temp_3['average_deaths_2015_2019_all_ages']

ax0.fill_between(temp_3['date'], y1, y2, where=(y1 > y2), color='#9b1b30', alpha=0.7,interpolate=True)

ax0.text(Xstart+10,Yend+2500,"Excess deaths [Daily]",fontsize=20,fontweight='bold')

ax0.text(Xend-255,list(temp_3['deaths_2020_all_ages'])[-1]+10000,'2020 Deaths',color='#0f4c81',fontweight='bold',fontsize=8, rotation=0)    
ax0.text(Xend-250,list(temp_3['average_deaths_2015_2019_all_ages'])[-1]-2500,'Avg. Deaths 2015-2020',color='gray',fontweight='bold',fontsize=8, rotation=0)
    
plt.show()

One can see when the pandemic really started to hit the UK. 

Early 2020 was a period of high death. If the data was available for 2021, I am sure we'd see an even larger red area, representing an even greater gap between the experienced deaths and those expected.

We can also view these differences **cumulatively**, to see the absolute differences across the course of the year.

In [ ]:
fig = plt.figure(figsize=(10, 3), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

for s in ["right", "top"]:
    ax0.spines[s].set_visible(False)
    
temp_3 = uk.loc['2020-01-01':]

sns.lineplot(x='date',y=temp_3['deaths_2020_all_ages'].cumsum(),data=temp_3,color='#0f4c81', ax=ax0)
sns.lineplot(x='date',y=temp_3['average_deaths_2015_2019_all_ages'].cumsum(),data=temp_3,color='gray', ax=ax0)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=3, maxticks=15)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart+10,Xend-30)

ax0.set_ylabel("Cumulative Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)

y1 = temp_3['deaths_2020_all_ages'].cumsum()
y2 = temp_3['average_deaths_2015_2019_all_ages'].cumsum()

ax0.fill_between(temp_3['date'], y1, y2, where=(y1 > y2), color='#9b1b30', alpha=0.7,interpolate=True)

ax0.text(Xstart+10,Yend+100000,"Excess deaths [Cumulative]",fontsize=20,fontweight='bold')

ax0.text(Xend-165,list(temp_3['deaths_2020_all_ages'].cumsum())[-1]-100000,'2020 Deaths',color='#0f4c81',fontweight='bold',fontsize=8, rotation=0)    
ax0.text(Xend-140,list(temp_3['average_deaths_2015_2019_all_ages'].cumsum())[-1]-300000,'Avg. Deaths 2015-2020',color='gray',fontweight='bold',fontsize=8, rotation=0)
    
plt.show()

When viewed like this, one can see that the primary increase came in April/May, as we saw earlier, but then the deaths tracked closely to the 5-year average.

To view these differences explicitly, I'll retrieve the exact figures.

We see a total difference of **around 80,000 additional deaths** in 2020 when compared to the 5-year average.
~

In [ ]:
num_2020 = round(y1.max())
num_avg = round(y2.max())

daily_num_2020 = round(y1.max()/365)
daily_num_avg = round(y2.max()/365)

fig = plt.figure(figsize=(6,1.4),dpi=150)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0.2, hspace=0.4)
ax0 = fig.add_subplot(gs[0, 0])

fig.patch.set_facecolor(background_color) # figure background color
ax0.set_facecolor(background_color) 


ax0.text(0,0.6,'{:,}'.format(daily_num_2020),color='#0f4c81',fontsize=25, fontweight='bold',ha='center')
ax0.text(0,0.25,"2020 daily average",color='#0f4c81',fontsize=15,ha='center')

ax0.text(0.77,0.6,'{:,}'.format(daily_num_avg),color='gray',fontsize=25, fontweight='bold',ha='center')
ax0.text(0.75,0.25,"2015-2019 daily average",color='gray',fontsize=15,ha='center')


ax0.text(0,-0.66,'{:,}'.format(num_2020),color='#0f4c81',fontsize=25, fontweight='bold',ha='center')
ax0.text(0,-0.25,"2020 total",color='#0f4c81',fontsize=15,ha='center')

ax0.text(0.77,-0.66,'{:,}'.format(num_avg),color='gray',fontsize=25, fontweight='bold',ha='center')
ax0.text(0.75,-0.25,"2015-2019 total average",color='gray',fontsize=15,ha='center')

ax0.set_yticklabels('')
ax0.set_xticklabels('')
ax0.tick_params(axis='both',length=0)

for s in ['top','right','left','bottom']:
    ax0.spines[s].set_visible(False)
    
import matplotlib.lines as lines
l1 = lines.Line2D([-0.1, 1], [0.155, 0.155], transform=fig.transFigure, figure=fig,color = 'gray', linestyle='-',linewidth = 0.7, alpha = .3)
fig.lines.extend([l1])
    
plt.show()

We have now expored the situation in some death in the United Kingdom.

We've looked at:

* Deaths in 2020 and the beginning of 2021

* Excess deaths 

* Cumulative differences


Now we have gained an understanding of the landscape in the United Kingdom, I will introduce an additional variable which might be able to act as an indicator for an incoming COVID-19 wave


# Introducing Google Searches

Now is the time we ask ourselves the question, **'can Google searches act as an early warning signal that cases, and deaths, are about to increase?'**

**I won't be trying to predict how severe a wave might be, or how many deaths might occur**. Rather, I want to know if an increase in Google searches for particular terms can act as a signal, or early sign, that more people are getting infected, and therefore that there might be more deaths.

The Google Trends data is indexed to the all-time searches for that particular term. 
For example, a score of 100 means that it is the highest it has ever been, and all other scores are relative to that number.

In [ ]:
temp_daily = uk_daily.groupby(['date','location'])['new_deaths_smoothed'].sum().unstack()

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

sns.lineplot(x='date',y='United Kingdom',data=temp_daily,color='#0F4C81',ax=ax0)

ax1 = plt.twinx()
sns.lineplot(x='Week',y='loss of smell: (United Kingdom)',data=trend_2020,color='#7F6917', ax=ax1)


Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart,Xend-1)
ax0.set_ylabel("Daily Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)

ax1.set_ylabel("Google searches for 'Loss of Smell' in the UK",fontsize=8,loc='top')
ax1.tick_params(axis = "both", which = "both", left=False, bottom=False, right=False)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=3, maxticks=12)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))


for s in ["top"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)


for s in ["left","bottom","right"]:
    ax0.spines[s].set_color(COLOR)
    
ax0.text(Xstart,Yend+200,"Daily deaths in the UK\nwith Google searches for 'loss of smell'",fontsize=20,fontweight='bold')


plt.show()

# Initial Oberservations

Above it is clear the there does appear to be a relationship between an increase in the search term 'Loss of smell' (a COVID-20 symptom) and a **subsequent rise in deaths.**

We can clearly see that in March 2020, 'Loss of smell' was searched for the most times. This may be due to people experiencing the symptom, and this be indicative of a rise in cases.

However, it could also simply be because this symptom had a lot of exposure on the news & interent, and so people were searching for it.

To try to figure out which of the explanations above it might be, let's move further in to the pandemic, say from June 2020 onwards. This should remove the initial peak in interest.

In [ ]:
temp_daily = uk_daily.groupby(['date','location'])['new_deaths_smoothed'].sum().unstack()
temp_daily = temp_daily.loc['2020-06-01':]

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

sns.lineplot(x='date',y='United Kingdom',data=temp_daily,color='#0f4c81',ax=ax0)

ax1 = plt.twinx()
temp2 = trend_2020.loc['2020-06-01':]
sns.lineplot(x='Week',y='loss of smell: (United Kingdom)',data=temp2,color='#7f6917', ax=ax1)


Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart,Xend-1)
ax0.set_ylabel("Daily Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)

ax1.set_ylabel("Google searches for 'Loss of Smell' in the UK",fontsize=8,loc='top')
ax1.tick_params(axis = "both", which = "both", left=False, bottom=False, right=False)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=3, maxticks=12)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

for s in ["top"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)


for s in ["left","bottom","right"]:
    ax0.spines[s].set_color(COLOR)
    

ax0.text(Xstart,Yend+200,"Daily deaths in the UK\nwith Google searches for 'loss of smell'",fontsize=20,fontweight='bold')

plt.show()

# Observations

Now we have removed the first few months of the pandemic, the relationship is arguably even easier to spot.

It seems that **when there is an increase in the frequency of people searching for 'Loss of smell'**, after delay of approximately one month, **there is a subsequent increase in recorded deaths.**


I will now **smooth** the search terms by plotting the 4 week rolling average. This should help to see underlying trends, rather easier than in the quite erratic plot above

In [ ]:
temp2['roll'] = temp2[['Week','loss of smell: (United Kingdom)']].rolling(4).mean()

temp_daily = uk_daily.groupby(['date','location'])['new_deaths_smoothed'].sum().unstack()
temp_daily = temp_daily.loc['2020-06-01':]

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor(background_color)

sns.lineplot(x='date',y='United Kingdom',data=temp_daily,color='#0f4c81',ax=ax0)

ax1 = plt.twinx()
#temp2 = trend_2020.loc['2020-06-01':]
sns.lineplot(x='Week',y='roll',data=temp2,color='#7f6917', ax=ax1)


Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_ylim([0,Yend])
ax0.set_xlim(Xstart,Xend-1)
ax0.set_ylabel("Daily Deaths",fontsize=8,loc='top')
ax0.set_xlabel(" ",fontsize=8,loc='left')
ax0.tick_params(axis = "both", which = "both", left=False, bottom=False)

ax1.set_ylabel("Google searches for 'Loss of Smell' in the UK",fontsize=8,loc='top')
ax1.tick_params(axis = "both", which = "both", left=False, bottom=False, right=False)


plt.setp(ax0.xaxis.get_majorticklabels(), rotation = 0)
locator = mdates.AutoDateLocator(minticks=5, maxticks=12)
formatter = mdates.ConciseDateFormatter(locator)
ax0.xaxis.set_major_locator(locator)
ax0.xaxis.set_major_formatter(formatter)
ax0.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

for s in ["top"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)


for s in ["left","bottom","right"]:
    ax0.spines[s].set_color(COLOR)
    

ax0.text(Xstart,Yend+200,"Daily deaths in the UK\nwith Google searches for 'loss of smell'",fontsize=20,fontweight='bold')

plt.show()

This is arguably the best plot to view the relationship we have been trying to describe.

**An increase in Google Searches for 'Loss of smell' (a COVID-19 symptom) appears to be indicative of a subsequent increase in recorded deaths.**

I hypothesise that this is the case because as more people start to get infected, and suffer symptoms, they begin 'googling' what it is they are experiencing. 

As there is a delay between infection and subsequent death, this is not reflected instantly in the data. Rather, the delay is also witnessed. 

 # **Why does this matter?**

This could be important for governments and local health authorities around the world, **particularly if a given country does not have adequate COVID-19 testing facilities** to be able to track the spread of the virus. 

A government or local health authority **could monitor Google searches for a variety of key words or phrases**, for instance all symptoms associated with COVID-19, and perhaps those associated with similar viruses such as Influenza, and if an increase in any of those search terms is detected, **action could be taken to mitigate further spread, and to prepare for a potential increase in hospitalisations**.



# work in progress 

There is more for me to add on this, such as **more search terms** which may be even more indicative of COVID-19 infection.

I would also like to expand this research in to **other countries, such as India**. Of course, this depends on how widespread Google is used, but it would be worth investigating.

We could also bring in **vaccinations**. Can Google search trends help to understand vaccine sentiment, or vaccine uptake for instance?